# Exercise 1 Solution: Build Your First Stateful Agent

This notebook implements a stateful agent with three custom tools:
1. **Weather tool:** Returns simulated weather for a given city
2. **Dictionary tool:** Looks up word definitions
3. **Web search tool:** Uses DuckDuckGo to search the web


## Setup and Installation

In [ ]:
# Install required packages
!pip install -q langgraph langchain langchain-openai python-dotenv duckduckgo-search

In [ ]:
# Imports
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Literal
from duckduckgo_search import DDGS
import os
import random

print("✅ All imports successful")

In [ ]:
# Load API key
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found! Please set it in your .env file.")

print("✅ API key loaded")

In [ ]:
# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    api_key=openai_api_key
)

print(f"✅ LLM initialized: {llm.model_name}")

## Tool 1: Weather Tool

Returns simulated weather information for any city.

In [ ]:
@tool
def get_weather(city: str) -> str:
    """
    Get the current weather for a given city.
    Use this tool when the user asks about weather conditions, temperature, or forecast.
    
    Args:
        city: The name of the city to get weather for (e.g., "Lagos", "New York", "Tokyo")
        
    Returns:
        Current weather information including temperature, conditions, and humidity
        
    Examples:
        - "Lagos" returns weather info for Lagos
        - "New York" returns weather info for New York
    """
    # Simulate weather data
    conditions = ["Sunny", "Cloudy", "Rainy", "Partly Cloudy", "Thunderstorms", "Clear"]
    temperature = random.randint(20, 35)  # Temperature in Celsius
    humidity = random.randint(40, 90)
    condition = random.choice(conditions)
    
    return f"""Weather in {city.title()}:
🌡️ Temperature: {temperature}°C
☁️ Conditions: {condition}
💧 Humidity: {humidity}%
🌬️ Wind: {random.randint(5, 25)} km/h"""

print("✅ Weather tool created")

In [ ]:
# Test weather tool
result = get_weather.invoke({"city": "Lagos"})
print(result)

## Tool 2: Dictionary Tool

Looks up word definitions from a predefined dictionary.

In [ ]:
@tool
def lookup_definition(word: str) -> str:
    """
    Look up the definition of a word.
    Use this tool when the user asks for word meanings, definitions, or explanations of terms.
    
    Args:
        word: The word to define (e.g., "ephemeral", "serendipity", "ubiquitous")
        
    Returns:
        The definition of the word, or a message if the word is not found
        
    Examples:
        - "ephemeral" returns the definition of ephemeral
        - "serendipity" returns the definition of serendipity
    """
    # Simulated dictionary with common words
    dictionary = {
        "ephemeral": "lasting for a very short time; transient",
        "serendipity": "the occurrence of events by chance in a happy or beneficial way",
        "ubiquitous": "present, appearing, or found everywhere",
        "paradigm": "a typical example or pattern of something; a model",
        "eloquent": "fluent or persuasive in speaking or writing",
        "ambiguous": "open to more than one interpretation; not having one obvious meaning",
        "benevolent": "well meaning and kindly",
        "cognitive": "related to the mental processes of perception, memory, judgment, and reasoning",
        "diligent": "having or showing care and conscientiousness in one's work or duties",
        "enigma": "a person or thing that is mysterious, puzzling, or difficult to understand",
        "fastidious": "very attentive to and concerned about accuracy and detail",
        "gregarious": "fond of company; sociable",
        "hackneyed": "lacking significance through having been overused; unoriginal and trite",
        "idiosyncratic": "relating to idiosyncrasy; peculiar or individual",
        "juxtapose": "place or deal with close together for contrasting effect"
    }
    
    word_lower = word.lower().strip()
    
    if word_lower in dictionary:
        return f"📖 Definition of '{word}':\n\n{dictionary[word_lower]}"
    else:
        return f"❌ Sorry, I don't have a definition for '{word}' in my dictionary. Try using web search for this word!"

print("✅ Dictionary tool created")

In [ ]:
# Test dictionary tool
result = lookup_definition.invoke({"word": "ephemeral"})
print(result)

## Tool 3: Web Search Tool

Uses DuckDuckGo to search the web for current information.

In [ ]:
@tool
def web_search(query: str) -> str:
    """
    Search the web using DuckDuckGo to find current information.
    Use this tool when you need to find recent information, news, or facts not in your knowledge base.
    
    Args:
        query: The search query (e.g., "latest AI news", "Python tutorials", "current events")
        
    Returns:
        Top search results with titles, snippets, and URLs
        
    Examples:
        - "latest AI news" searches for recent AI developments
        - "Python best practices" searches for Python programming resources
    """
    try:
        # Perform DuckDuckGo search
        ddgs = DDGS()
        results = ddgs.text(query, max_results=5)
        
        if not results:
            return f"No results found for: {query}"
        
        # Format results
        formatted_results = f"🔍 Search results for '{query}':\n\n"
        
        for i, result in enumerate(results, 1):
            title = result.get('title', 'No title')
            snippet = result.get('body', 'No description')
            url = result.get('href', '')
            
            formatted_results += f"{i}. **{title}**\n"
            formatted_results += f"   {snippet[:200]}...\n"
            formatted_results += f"   🔗 {url}\n\n"
        
        return formatted_results
        
    except Exception as e:
        return f"❌ Error performing web search: {str(e)}"

print("✅ Web search tool created")

In [ ]:
# Test web search tool
result = web_search.invoke({"query": "LangGraph tutorials"})
print(result[:500])  # Print first 500 chars

## Build the Agent Graph

Now let's create the stateful agent that can use all three tools.

In [ ]:
# Create tools list
tools = [get_weather, lookup_definition, web_search]

# Bind tools to LLM
llm_with_tools = llm.bind_tools(tools)

print(f"✅ LLM bound to {len(tools)} tools")
print(f"   Tools: {[tool.name for tool in tools]}")

In [ ]:
# System prompt
sys_msg = SystemMessage(content="""You are a helpful assistant with access to multiple tools:

1. **get_weather**: Use when asked about weather, temperature, or climate in any city
2. **lookup_definition**: Use when asked to define words or explain terms
3. **web_search**: Use when you need current information, news, or facts from the internet

Guidelines:
- Choose the most appropriate tool for each query
- For simple questions you can answer directly, don't use tools
- Be concise and helpful in your responses
- If a tool returns an error, acknowledge it and suggest alternatives""")

def assistant(state: MessagesState) -> dict:
    """
    Assistant node - decides whether to use tools or answer directly.
    """
    messages = [sys_msg] + state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

print("✅ Assistant node defined")

In [ ]:
# Conditional routing function
def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """
    Decide next step based on last message.
    
    If LLM called a tool → go to 'tools' node
    If LLM provided final answer → go to END
    """
    last_message = state["messages"][-1]
    
    # Check if LLM made tool calls
    if last_message.tool_calls:
        return "tools"
    
    # No tool calls - we're done
    return "__end__"

print("✅ Conditional routing function defined")

In [ ]:
# Build the graph
builder = StateGraph(MessagesState)

# Add nodes
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    should_continue,
    {"tools": "tools", "__end__": END}
)
builder.add_edge("tools", "assistant")

# Add memory
memory = MemorySaver()
agent = builder.compile(checkpointer=memory)

print("✅ Agent graph compiled with 3 tools and memory")

In [ ]:
# Visualize the graph
try:
    display(Image(agent.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Could not display graph: {e}")
    print("Graph structure: START → assistant → [conditional] → tools → assistant → END")

## Testing the Agent

Let's test all the example queries from the exercise.

In [ ]:
# Helper function
def run_agent(user_input: str, thread_id: str = "test_session"):
    """
    Run the agent and display the conversation.
    """
    print(f"\n{'='*70}")
    print(f"👤 User: {user_input}")
    print(f"{'='*70}\n")
    
    result = agent.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config={"configurable": {"thread_id": thread_id}}
    )
    
    for message in result["messages"]:
        if isinstance(message, HumanMessage):
            continue
        elif isinstance(message, AIMessage):
            if message.tool_calls:
                tool_name = message.tool_calls[0]['name']
                print(f"🤖 Agent: [Calling tool: {tool_name}]")
            else:
                print(f"🤖 Agent: {message.content}")
        elif isinstance(message, ToolMessage):
            content = message.content
            if len(content) > 300:
                print(f"🔧 Tool Result: {content[:300]}...")
            else:
                print(f"🔧 Tool Result: {content}")
    
    print(f"\n{'='*70}\n")

print("✅ Test function ready")

### Test 1: Weather Query

In [ ]:
run_agent("What's the weather in Lagos?")

### Test 2: Dictionary Query

In [ ]:
run_agent("Define the word 'ephemeral'")

### Test 3: Web Search Query

In [ ]:
run_agent("Search for latest AI news")

### Test 4: No Tool Needed

In [ ]:
run_agent("What's the capital of France?")

### Test 5: Multiple Queries in Same Session

In [ ]:
# First query
run_agent("What's the weather in Tokyo?", thread_id="multi_session")

In [ ]:
# Follow-up query
run_agent("Now search for best restaurants in that city", thread_id="multi_session")

### Test 6: Word Not in Dictionary

In [ ]:
run_agent("Define the word 'antidisestablishmentarianism'")

### Test 7: Complex Query Requiring Agent Decision

In [ ]:
run_agent("I want to know about serendipity and the weather in London")

## Summary

### ✅ Exercise Requirements Completed

1. **Three Custom Tools Created:**
   - ✅ Weather tool (simulated weather data)
   - ✅ Dictionary tool (word definitions)
   - ✅ Web search tool (DuckDuckGo integration)

2. **Tool Integration:**
   - ✅ Tools defined using `@tool` decorator
   - ✅ Tools bound to LLM using `bind_tools()`
   - ✅ Clear docstrings for each tool

3. **Conditional Routing:**
   - ✅ Agent decides which tool to use
   - ✅ Handles cases where no tool is needed
   - ✅ Loops back to assistant after tool execution

4. **State Management:**
   - ✅ Memory persists across conversation turns
   - ✅ Agent remembers context from previous queries

### 🎯 Key Learnings

- Agents autonomously decide which tool to use based on query
- Multiple tools can coexist and be intelligently selected
- Conditional routing enables iterative tool usage
- Web search integration provides real-time information
- Error handling ensures graceful failures

### 🚀 Next Steps

Ready for **Topic 3: Agentic RAG** where retrieval becomes a tool!